## This notebook can be used to create ETS components, SARIMAX, and shortlist of non-media drivers without having to run the other functionalities of automation_step1_variable_selection

#### Import libraries
* Custom modules used:
    * custom_variable_transformation: module with class to handle media transformations
    * custom_seasonality_analysis_updated: module with class to handle seasonality with residuals of "core" (christmas/thanksgiving, weekend, best price and best inventory) variables
    * custom_transformation_functions: module with common transformation functions
    * eda_visualization: reusing eda class for plotting

In [ ]:
# Dataframe and array manipulation
import pandas as pd
import numpy as np
import os
import re

#Linear regression
import statsmodels.api as sm

#Computation of Spearman correlation clusters
from scipy import stats
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
from collections import defaultdict
plt.rcParams['figure.dpi'] = 300
plt.rcParams["figure.figsize"] = (20, 20)

# Custom Libraries for saving excel and transformation of variables
import sys
module_path = os.path.abspath(os.path.join('../../src/utils'))
if module_path not in sys.path:
    sys.path.append(module_path)

from custom_seasonality_analysis import VariableSeasonality

import warnings
warnings.filterwarnings('ignore')

## User inputs

#### Input outcome variables, time and path to stack

In [ ]:
#User Input: Outcome variables for the two models
outcome_variable_units='O_UNIT'
outcome_variable_newcust='O_NEW_CUSTOMER'
#User Input: Modeling time frame
modeling_start='2019-01-01'
modeling_end='2021-04-30'

In [ ]:
# User input: Path to the modeling stack dataframe
input_file_path = "../../data/input_data/modeling_stack.csv" 

#### Input essential non-media variables for each outcome

Add any essential time-based dummies here, your EDA should help here: Christmas and Thanksgiving + weekends are a minimum. Black Friday, COVID peak, NYE, etc. could also be added.

In [ ]:
#User input: Add any important time and non-negotiable, e.g., from EDA + client requested variables. 
#Christmas and Thanksgiving + weekends are a minimum. 
essential_dummies_units=['D_HOL_CHRISTMAS','D_HOL_THANKSGIVING','D_DAY_SATURDAY',
                         'D_DAY_SUNDAY']
essential_dummies_newcust=['D_HOL_CHRISTMAS','D_HOL_THANKSGIVING','D_DAY_SATURDAY',
                           'D_DAY_SUNDAY']

#### Input seasonality method for each model

This can be a subset of SARIMAX, SMA, SEASONALITY, TREND, RESIDUAL, SARIMAX_LOG, SEASONALITY_LOG, TREND_LOG, and RESIDUAL_LOG. Default is ['SARIMAX'] for both models.

In [ ]:
#User input: Modify seasonality term to be considered if you believe SARIMAX is too restrictive. 
seasonality_terms_units=['SARIMAX']
seasonality_terms_newcust=['SARIMAX']

In [ ]:
#User input: Set False if you would like to revert to old behavior where seasonality terms were fitted
#to the raw output instead of residuals of output to essential dummies and price/distribution.
seasonality_fit_resid_units=True
seasonality_fit_resid_newcust=True

### Loading the dataframe

In [ ]:
df = pd.read_csv(input_file_path)
df['index'] = pd.to_datetime(df['index']) # Formatting the date
df.set_index("index", inplace = True, drop= False) 


#### User Input: Add any custom variables to be generated on df; e.g.,

In [ ]:
df['D_EVENT_JAN3']=0
df.loc['2021-01-03','D_EVENT_JAN3']=1

#### Check if stated variables are in dataframe

In [ ]:
#Warns user if stated variables not in dataframe
for dummy in essential_dummies_units:
    if dummy not in df.columns:
        print(dummy, ' deemed essential for units model not in modeling stack.')
for dummy in essential_dummies_newcust:
    if dummy not in df.columns:
        print(dummy, ' deemed essential for units model not in modeling stack.')

In [ ]:
type(seasonality_fit_resid_units)==bool

In [ ]:
#Options validation for seasonality_terms and seasonality_fit_resid
valid_options_seas=['SARIMAX', 'SMA', 'SEASONALITY', 'TREND', 'RESIDUAL',
                    'SARIMAX_LOG', 'SEASONALITY_LOG', 'TREND_LOG', 'RESIDUAL_LOG']
valid_options_fit_resid=[True,False]
if not all(x in valid_options_seas for x in seasonality_terms_units):
    raise ValueError("""Invalid options entered for seasonality_terms_units. Inputs must be a subset of
                     ['SARIMAX', 'SMA', 'SEASONALITY', 'TREND', 'RESIDUAL',
                    'SEASONALITY', 'SARIMAX_LOG', 'SEASONALITY_LOG', 'TREND_LOG', 'RESIDUAL_LOG']""")
if not type(seasonality_fit_resid_units)==bool:
    raise ValueError("""Invalid option entered for seasonality_fit_resid_units. Input must be a boolean""")
if not type(seasonality_fit_resid_newcust)==bool:
    raise ValueError("""Invalid option entered for seasonality_fit_resid_newcust. Input must be a boolean""")

Filter the dataset on the specified data period

In [ ]:
df_modeling = df[(df.index >= modeling_start) & (df.index <= modeling_end)].copy()
#Remove near zero variance essential dummies, if any
essential_dummies_units=[c for c in essential_dummies_units if np.std(df_modeling[c])>1e-8]
essential_dummies_newcust=[c for c in essential_dummies_newcust if np.std(df_modeling[c])>1e-8]

Find the essential  price variable: Select the most negatively Pearson correlated price variable with the outcome

In [ ]:
def find_outcome_residuals(df,outcome_variable,essential_vars):
    '''
    Calculates the residual of outcome_variable regressed against essential_vars
    '''
    if len(essential_vars)>0:
        Y = df[outcome_variable]
        X = df[essential_vars]
        X = sm.add_constant(X)
        model = sm.OLS(Y, X, missing='drop')
        model_result = model.fit()
        outcome_residuals=model_result.resid
    else:
        outcome_residuals=df[outcome_variable]
    return outcome_residuals

In [ ]:
def find_essential_price_var(df,outcome_variable,essential_vars):
    '''
    Finds the lowest (i.e., most negative) Pearson correlated price variable to the outcome data
    '''
    outcome_residuals=find_outcome_residuals(df,outcome_variable,essential_vars)
    price_vars=[c for c in df.columns if 'PRICE' in c and 'O_' not in c]
    price_corrs=df[price_vars].join(outcome_residuals.rename('resids')).corr().iloc[-1] 
    price_corr=price_corrs[np.argmin(price_corrs)]
    if price_corr<0:
        price_var=[price_corrs.index[np.argmin(price_corrs)]]
        print('Price variable for '+outcome_variable+':',price_var,price_corr)
    else:
        price_var=[]
        print('No negatively Pearson correlated price variable found for '+outcome_variable+'.')
    return price_var
price_var_units=find_essential_price_var(df_modeling,outcome_variable_units,essential_dummies_units)
price_var_newcust=find_essential_price_var(df_modeling,outcome_variable_newcust,essential_dummies_newcust)

Find essential non-media variables: Select the most positively Pearson correlated inventory variable to the residuals of the outcome with price

In [ ]:
essential_vars_units=essential_dummies_units+price_var_units
essential_vars_newcust=essential_dummies_newcust+price_var_newcust

In [ ]:
def find_essential_inv_var(df,outcome_variable,essential_vars):
    '''
    Finds the highest Pearson correlated inventory/distribution variable (OOS variables are internally negated to
    be assed through the same condition) to the outcome data. 
    '''
    outcome_residuals=find_outcome_residuals(df,outcome_variable,essential_vars)      
    inventory_vars=[c for c in df.columns if 'INVENTORY' in c or 'TDP' in c or 'ACV' in c or 'DISTRIBUTION' in c]
    inv_corrs=df[inventory_vars].join(outcome_residuals.rename('resids')).copy()
    oos_cols=[c for c in inv_corrs.columns if 'OOS' in c]
    if len(oos_cols)>0:
        inv_corrs[oos_cols]=-inv_corrs[oos_cols]
    inv_corrs=inv_corrs.corr().iloc[-1]
    inv_corrs=inv_corrs[inv_corrs<1.0]
    inv_corr=inv_corrs[np.argmax(inv_corrs)]
    if inv_corr>0:
        inv_var=[inv_corrs.index[np.argmax(inv_corrs)]]
        if 'OOS' in inv_var:
            inv_corr=-inv_corr
        print('Price variable for '+outcome_variable+':',inv_var,inv_corr)
    else:
        inv_var=[]
        print('No positively Pearson correlated inventory variable found for '+outcome_variable+'.')
    return inv_var
inv_var_units=find_essential_inv_var(df_modeling,outcome_variable_units,essential_vars_units)
inv_var_newcust=find_essential_inv_var(df_modeling,outcome_variable_newcust,essential_vars_newcust)

In [ ]:
essential_vars_units=essential_dummies_units+price_var_units+inv_var_units
essential_vars_newcust=essential_dummies_newcust+price_var_newcust+inv_var_newcust

Check if variables are of expected sign.

In [ ]:
Y = df_modeling[outcome_variable_units]
X = df_modeling[essential_vars_units]
X = sm.add_constant(X)
model = sm.OLS(Y, X, missing='drop')
model_result = model.fit()
model_result.summary()

In [ ]:
Y = df_modeling[outcome_variable_newcust]
X = df_modeling[essential_vars_newcust]
X = sm.add_constant(X)
model = sm.OLS(Y, X, missing='drop')
model_result = model.fit()
model_result.summary()

### Add Trend & Seasonality to the residuals of essential variables

#### Add to New Customer Count

In [ ]:
if seasonality_fit_resid_newcust:
    seas_obj_cust = VariableSeasonality(df, key_variable=outcome_variable_newcust, exog=essential_vars_newcust)
else:
    seas_obj_cust = VariableSeasonality(df, key_variable=outcome_variable_newcust)

In [ ]:
# custom_order and custom_seasonal_order would overwrite the order from autom arima
# these are optional parameters
seas_obj_cust.arma_arima_func(start_p= 1, start_q= 1, day_start= 5, day_end= 8)

In [ ]:
# additional_period_start_date is optional parameter. for this period, ETS terms would be created separately
seas_obj_cust.creating_var_func()

In [ ]:
# Addition of the seasonality variables to the actual dataset
df_model_valid = seas_obj_cust.df

#### Add to Volume Sales Units

In [ ]:
if seasonality_fit_resid_units:
    seas_obj_units = VariableSeasonality(df_model_valid, key_variable=outcome_variable_units, exog=essential_vars_units)
else:
    seas_obj_units = VariableSeasonality(df_model_valid, key_variable=outcome_variable_units)    

In [ ]:
seas_obj_units.arma_arima_func(start_p= 1, start_q= 1, day_start= 5, day_end= 8)

In [ ]:
seas_obj_units.creating_var_func()

In [ ]:
df_model_valid = seas_obj_units.df

In [ ]:
df_modeling = df_model_valid[(df_model_valid.index >= modeling_start) & (df_model_valid.index <= modeling_end)].copy()

Choose correct seasonality variables using appropriate regex

In [ ]:
regex_seasonality_units=['_'+i+'$' for i in seasonality_terms_units if 'SARIMAX' not in i]+['_'+i+' \(' for i in seasonality_terms_units if 'SARIMAX' in i]
regex_seasonality_newcust=['_'+i+'$' for i in seasonality_terms_newcust if 'SARIMAX' not in i]+['_'+i+' \(' for i in seasonality_terms_newcust if 'SARIMAX' in i]
regex_seasonality_units='|'.join(regex_seasonality_units)
regex_seasonality_newcust='|'.join(regex_seasonality_newcust)

seasonality_vars_units=[c for c in df_modeling.columns if outcome_variable_units in c and re.search(regex_seasonality_units,c) is not None]
seasonality_vars_newcust=[c for c in df_modeling.columns if outcome_variable_newcust in c and re.search(regex_seasonality_newcust,c) is not None]

Check quality of regressions and signs when seasonality variables are added.

In [ ]:
Y = df_modeling[outcome_variable_units]
X = df_modeling[essential_vars_units+seasonality_vars_units]
X = sm.add_constant(X)
model = sm.OLS(Y, X, missing='drop')
model_result = model.fit()
model_result.summary()

In [ ]:
Y = df_modeling[outcome_variable_newcust]
X = df_modeling[essential_vars_newcust+seasonality_vars_newcust]
X = sm.add_constant(X)
model = sm.OLS(Y, X, missing='drop')
model_result = model.fit()
model_result.summary()

#### Generate stack with new seasonality variables in the processed_input_data folder
The dataframe would have addition additional seasonality, trends, SARIMAX terms.

In [ ]:
# Saving the stacked data (with additional variables)
df_model_valid.to_csv("../../data/processed_input_data/" + "02_modelling_stack_data_seasonality_autoselected.csv",
         index= False)

## Form a shortlist and long list of remaining non-media variables for further consideration

**Shortlist:** Pick one remaining media variable among remaining variables from each Spearman correlation cluster that doesn't have any essential variables (can have a tolerance>0), has the highest absolute Pearson correlation with the residual of outcome within all member variables of the cluster, then further eliminates variables that flips the expected signs of any other variables or has a VIF>2 (can be changed).

**Long list:** all conditions in shortlist except for the essential variable membership, coefficient flipping and VIF conditions.

Also reports a **variable ranking df** in the output_data folder, `output_data/modelling_output/new_customer/nonmedia_driver_ranking_newcust.csv` and `output_data/modelling_output/sales_unit/nonmedia_driver_ranking_units.csv` that lists the coefficient, R2, MAPE, VIF and any existing coefficient flippages of all remaining nonmedia drivers and orders them based on R2, MAPE, VIF and flippage ranking (R2 descending rank, others ascending rank since lower is better on those metrics) sum for that model. These metrics are calculated on a regression of essential_vars_newcust+variable versus outcome.

In [ ]:
essential_vars_newcust=essential_vars_newcust+seasonality_vars_newcust
essential_vars_units=essential_vars_units+seasonality_vars_units

In [ ]:
remaining_nonmedia_cols_newcust=[c for c in df_modeling.columns if 'M_' not in c 
                   and 'O_' not in c 
                   and 'index' not in c
                   and c not in essential_vars_newcust
                   and 'RATE' not in c
                   and np.std(df_modeling[c])>1e-8] #Also don't consider near-zero variance variables, if any
remaining_nonmedia_cols_units=[c for c in df_modeling.columns if 'M_' not in c 
                   and 'O_' not in c 
                   and 'index' not in c
                   and c not in essential_vars_units
                   and 'RATE' not in c
                   and np.std(df_modeling[c])>1e-8]

Function definitions, you don't need to edit anything here

In [ ]:
def find_variable_coef(df,outcome_variable,essential_vars,variable):
    '''
    Finds coefficient of added "variable" within a regression of outcome_variable~essential_vars+variable
    '''
    Y = df[outcome_variable]
    X = df[essential_vars+[variable]].copy()
    X = sm.add_constant(X)
    model = sm.OLS(Y.copy(), X.copy(), missing='drop')
    params=model.fit().params.copy()
    return params[variable]

In [ ]:
def find_variable_r2(df,outcome_variable,essential_vars,variable):
    '''
    Finds R2 of the regression of outcome_variable~essential_vars+variable
    '''
    Y = df[outcome_variable]
    X = df[essential_vars+[variable]]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X, missing='drop')
    return model.fit().rsquared

In [ ]:
def find_variable_mape(df,outcome_variable,essential_vars,variable):
    '''
    Finds MAPE of the regression of outcome_variable~essential_vars+variable
    '''
    Y = df[outcome_variable]
    X = df[essential_vars+[variable]]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X, missing='drop').fit()
    perc_error = abs((model.predict()/Y)-1)
    return perc_error.mean() * 100    

In [ ]:
def vif(df_X):
    '''
    Finds VIF of the regressors of outcome_variable~essential_vars+variable
    '''
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    #Calculate the VIF for each feature of df_X (a Pandas DataFrame).
    vif_data = [variance_inflation_factor(df_X.values, i)
        for i in range(df_X.shape[1])]
    return pd.Series(vif_data, index=df_X.columns)

In [ ]:
def find_variable_vif(df,essential_vars,variable):
    '''
    Extracts the variable VIF among variables of the regression of outcome_variable~essential_vars+variable
    '''
    return vif(df[essential_vars+[variable]][~df[variable].isnull()])[variable]

In [ ]:
def find_coef_flips(df,outcome_variable,essential_vars,variable):
    '''
    Finds the number of flipped coefficient signs among essential_vars
    within the regression of outcome_variable~essential_vars+variable
    compared with outcome_variable~essential_vars
    '''
    Y = df[outcome_variable]
    X_ess = df[essential_vars]
    X_ess = sm.add_constant(X_ess)
    model_ess = sm.OLS(Y.copy(), X_ess.copy(), missing='drop')
    params_ess=model_ess.fit().params.copy() 
    X = df[essential_vars+[variable]].copy()
    X = sm.add_constant(X)
    model = sm.OLS(Y.copy(), X.copy(), missing='drop')
    params_var=model.fit().params.copy()
    counter=0
    for var in essential_vars:
        if params_ess[var]*params_var[var]<0:
            counter+=1
    return counter

In [ ]:
def order_by_ranking(df):
    '''
    Orders additional variables by their ranking in final R2 (descending), MAPE, variable VIF and 
    other non-media variable Coefficient flips once added to the regresssion
    '''
    df['Variable Ranking']=df['R2'].rank(ascending=False)+df['MAPE'].rank()+df['VIF'].rank()+df['Coef Flips'].rank()
    df=df.sort_values(by=['Variable Ranking'])
    return df

### Auto produce remaining non-media driver rankings

Calculate and report the non-media driver ranking into an output file. You don't need to edit anything here. You can find the outputs in the paths to the calls to pd.DataFrame.to_csv()

In [ ]:
nonmedia_ranking_newcust=[]
nonmedia_ranking_units=[]
for var in remaining_nonmedia_cols_newcust:
    var_row={}
    var_row['Variable']=var
    var_row['Coefficient']=find_variable_coef(df_modeling,outcome_variable_newcust,essential_vars_newcust,var)
    var_row['R2']=find_variable_r2(df_modeling,outcome_variable_newcust,essential_vars_newcust,var)
    var_row['MAPE']=find_variable_mape(df_modeling,outcome_variable_newcust,essential_vars_newcust,var)
    var_row['VIF']=find_variable_vif(df_modeling,essential_vars_newcust,var)
    var_row['Coef Flips']=find_coef_flips(df_modeling,outcome_variable_newcust,essential_vars_newcust,var)
    nonmedia_ranking_newcust.append(var_row)
for var in remaining_nonmedia_cols_units:
    var_row={}
    var_row['Variable']=var
    var_row['Coefficient']=find_variable_coef(df_modeling,outcome_variable_units,essential_vars_units,var)
    var_row['R2']=find_variable_r2(df_modeling,outcome_variable_units,essential_vars_units,var)
    var_row['MAPE']=find_variable_mape(df_modeling,outcome_variable_units,essential_vars_units,var)
    var_row['VIF']=find_variable_vif(df_modeling,essential_vars_units,var)
    var_row['Coef Flips']=find_coef_flips(df_modeling,outcome_variable_units,essential_vars_units,var)
    nonmedia_ranking_units.append(var_row)
nonmedia_ranking_newcust=order_by_ranking(pd.DataFrame(nonmedia_ranking_newcust))
nonmedia_ranking_newcust.to_csv('../../data/output_data/modelling_output/new_customer/nonmedia_driver_ranking_newcust.csv')
nonmedia_ranking_units=order_by_ranking(pd.DataFrame(nonmedia_ranking_units))
nonmedia_ranking_units.to_csv('../../data/output_data/modelling_output/sales_unit/nonmedia_driver_ranking_units.csv')

In [ ]:
nonmedia_ranking_units.head()

### Visualize and find Spearman correlation clusters of remaining non-media drivers: New Customer

This plotting function visualizes the Spearman correlations of non-media drivers as a dendrogram and a heatmap. A quick obsevation typically shows that nonmedia drivers of similar semantics e.g., those that are related to distribution or competitive activity, usually are clustered in the same bucket because variables therein are highly correlated among each other.

In [ ]:
visualize_plots=False #Whether to visualize or not, default=False
df_newcust=df_modeling[remaining_nonmedia_cols_newcust+essential_vars_newcust]
current_predictors=df_newcust.columns
corr = spearmanr(df_newcust).correlation

# Ensure the Spearman correlation matrix is symmetric
corr = (corr + corr.T)/2
np.fill_diagonal(corr, 1)

# hierarchical clustering using Ward's linkage.
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
if visualize_plots:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
    dendro = hierarchy.dendrogram(
    dist_linkage, labels=current_predictors, ax=ax1, leaf_rotation=90
    )
    dendro_idx = np.arange(0, len(dendro['ivl']))
    ax2.imshow(corr[dendro['leaves'], :][:, dendro['leaves']])
    ax2.set_xticks(dendro_idx)
    ax2.set_yticks(dendro_idx)
    ax2.set_xticklabels(dendro['ivl'], rotation='vertical')
    ax2.set_yticklabels(dendro['ivl'])
    fig.tight_layout()
    plt.show()


cluster_ids_newcust = hierarchy.fcluster(dist_linkage, 1, criterion='distance')
cluster_id_to_feature_ids_newcust = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids_newcust):
    cluster_id_to_feature_ids_newcust[cluster_id].append(current_predictors[idx])

### Visualize and find Spearman correlation clusters of remaining non-media drivers: Unit Sales

This plotting function visualizes the Spearman correlations of non-media drivers as a dendrogram and a heatmap. A quick obsevation typically shows that nonmedia drivers of similar semantics e.g., those that are related to distribution or competitive activity, usually are clustered in the same bucket because variables therein are highly correlated among each other.

In [ ]:
visualize_plots=True #Whether to visualize or not, default=False
df_units=df_modeling[remaining_nonmedia_cols_units+essential_vars_units]
current_predictors=df_units.columns
corr = spearmanr(df_units).correlation

# Ensure the Spearman correlation matrix is symmetric
corr = (corr + corr.T)/2
np.fill_diagonal(corr, 1)

# hierarchical clustering using Ward's linkage.
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
if visualize_plots:
    fig, (ax1, ax2) = plt.subplots(2, 1,figsize=(20,20))
    dendro = hierarchy.dendrogram(
    dist_linkage, labels=current_predictors, ax=ax1, leaf_rotation=90
    )
    dendro_idx = np.arange(0, len(dendro['ivl']))
    ax2.imshow(corr[dendro['leaves'], :][:, dendro['leaves']])
    ax2.set_xticks(dendro_idx)
    ax2.set_yticks(dendro_idx)
    ax2.set_xticklabels(dendro['ivl'], rotation='vertical')
    ax2.set_yticklabels(dendro['ivl'])
    fig.tight_layout()
    plt.show()


cluster_ids_units = hierarchy.fcluster(dist_linkage, 1, criterion='distance')
cluster_id_to_feature_ids_units = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids_units):
    cluster_id_to_feature_ids_units[cluster_id].append(current_predictors[idx])

### Produce residual series

In [ ]:
resid_newcust=find_outcome_residuals(df_modeling,outcome_variable_newcust,essential_vars_newcust)
resid_units=find_outcome_residuals(df_modeling,outcome_variable_units,essential_vars_units)

### Functions for producing shortlist and long list

In [ ]:
def find_clust_most_correlated_variable(clusters,cluster_id,resid_series):
    '''
    Finds the variable most (Pearson) correlated with the outcome residuals within each cluster
    '''
    var_corrs={}
    for var in clusters[cluster_id]:
        corr=np.abs(pd.DataFrame({'var':df_modeling[var].copy(),'resids':resid_series})).copy().corr().iloc[0,1]
        var_corrs[var]=corr
    return max(var_corrs, key=var_corrs.get)

In [ ]:
def mark_essential_variable_clusters(clusters,essential_vars,tolerance=0):
    '''
    Finds if the cluster contains essential_vars beyond the tolerance
    '''
    contain_essential_var={}
    for cluster_id in clusters:
        if tolerance==0:
            val=any([var in essential_vars for var in clusters[cluster_id]])
        else:
            val=len([var for var in clusters[cluster_id] if var in essential_vars])>tolerance           
        contain_essential_var[cluster_id]=val
    return contain_essential_var

In [ ]:
def contain_flip_or_high_vif_vars(var,outcome_variable,VIF_tolerance=2,flip_tolerance=0):
    '''
    Finds if the cluster contains variables that can flip or seriously inflate VIF of the regression when
    added.
    '''
    if outcome_variable==outcome_variable_units:
        flip_or_high_vif=nonmedia_ranking_units[(nonmedia_ranking_units['VIF']>VIF_tolerance)|(nonmedia_ranking_units['Coef Flips']>flip_tolerance)].copy()
    elif outcome_variable==outcome_variable_newcust:
        flip_or_high_vif=nonmedia_ranking_newcust[(nonmedia_ranking_newcust['VIF']>VIF_tolerance)|(nonmedia_ranking_newcust['Coef Flips']>flip_tolerance)].copy()
    else:
        raise ValueError("Need to choose units or newcust outcome variable as outcome_variable")
    flip_or_high_vif_vars=flip_or_high_vif.Variable.tolist()
    val=(var in flip_or_high_vif_vars)
    return val

In [ ]:
essential_var_clusts_newcust=mark_essential_variable_clusters(cluster_id_to_feature_ids_newcust,essential_vars_newcust)
essential_var_clusts_units=mark_essential_variable_clusters(cluster_id_to_feature_ids_units,essential_vars_units)

In [ ]:
most_corrvars_newcust={cluster_id:find_clust_most_correlated_variable(cluster_id_to_feature_ids_newcust,cluster_id,resid_newcust)
                 for cluster_id in cluster_id_to_feature_ids_newcust.keys()}
most_corrvars_units={cluster_id:find_clust_most_correlated_variable(cluster_id_to_feature_ids_units,cluster_id,resid_units)
                 for cluster_id in cluster_id_to_feature_ids_units.keys()}

In [ ]:
longlist_newcust=[var for var in most_corrvars_newcust.values() if var not in essential_vars_newcust and var in remaining_nonmedia_cols_newcust]
longlist_units=[var for var in most_corrvars_units.values() if var not in essential_vars_units and var in remaining_nonmedia_cols_units]

In [ ]:
#Can change shortlist tolerances
shortlist_newcust=[var for id,var in most_corrvars_newcust.items() 
                   if not mark_essential_variable_clusters(cluster_id_to_feature_ids_newcust,essential_vars_newcust,tolerance=0)[id]
                   and not contain_flip_or_high_vif_vars(var,outcome_variable_newcust,VIF_tolerance=2,flip_tolerance=0)
                  and var in remaining_nonmedia_cols_newcust]
shortlist_units=[var for id,var in most_corrvars_units.items() 
                   if not mark_essential_variable_clusters(cluster_id_to_feature_ids_units,essential_vars_units,tolerance=0)[id]
                   and not contain_flip_or_high_vif_vars(var,outcome_variable_units,VIF_tolerance=2,flip_tolerance=0)
                and var in remaining_nonmedia_cols_units]

In [ ]:
#Append shortlist/longlist status and rewrite non-media variable rankings
nonmedia_ranking_newcust['Longlisted']=nonmedia_ranking_newcust.Variable.isin(longlist_newcust)
nonmedia_ranking_newcust['Shortlisted']=nonmedia_ranking_newcust.Variable.isin(shortlist_newcust)
nonmedia_ranking_units['Longlisted']=nonmedia_ranking_units.Variable.isin(longlist_units)
nonmedia_ranking_units['Shortlisted']=nonmedia_ranking_units.Variable.isin(shortlist_units)

### Remaining Non-Media Driver Rankings
The final output of the notebook is a variable ranking df in the output_data folder, `output_data/modelling_output/new_customer/nonmedia_driver_ranking_newcust.csv` and `output_data/modelling_output/sales_unit/nonmedia_driver_ranking_units.csv` that lists, along with longlisted and shortlisted status, the coefficient, R2, MAPE, VIF and any existing coefficient flippages of all remaining non-media drivers and orders them based on R2, MAPE, VIF and flippage ranking (R2 descending rank, others ascending rank since lower is better on those metrics) sum for that model. These metrics are calculated on a regression of outcome versus fixed variable+variable of interest.

In [ ]:
nonmedia_ranking_newcust.to_csv('../../data/output_data/modelling_output/new_customer/nonmedia_driver_ranking_newcust.csv')
nonmedia_ranking_units.to_csv('../../data/output_data/modelling_output/sales_unit/nonmedia_driver_ranking_units.csv')

In [ ]:
nonmedia_ranking_newcust

In [ ]:
nonmedia_ranking_units

### Final Regression Visualization with all shortlist variables

Show regressions with all shortlisted variables (a suggestion to start modeling---you may choose to remove those from the shortlist that are too down the ranking lists); check if all variables are of expected sign.

In [ ]:
Y = df_modeling[outcome_variable_units]
X = df_modeling[essential_vars_units+shortlist_units]
X = sm.add_constant(X)
model = sm.OLS(Y, X, missing='drop')
model_result = model.fit()
model_result.summary()

In [ ]:
Y = df_modeling[outcome_variable_newcust]
X = df_modeling[essential_vars_newcust+shortlist_newcust]
X = sm.add_constant(X)
model = sm.OLS(Y, X, missing='drop')
model_result = model.fit()
model_result.summary()